In [1]:
import valurap.commands as vac
import valurap.printer as vp
import valurap.axes as vax
import valurap.apgs as vap
import valurap.asg as vas
import valurap.spi as vs

import valurap.path_planning as vapp

from importlib import reload
reload(vs)
reload(vac)
reload(vas)
reload(vax)
reload(vap)
reload(vp)
reload(vapp)

prn = vp.Valurap()
prn.setup()
prn.axe_e2.max_v=20000*3
prn.axe_e1.max_v=20000*22

/home/seva/.virtualenvs/valurap3/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/home/seva/.virtualenvs/valurap3/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


Executing reset code
Update axes
Reset OLED
Setup done


In [2]:
prn.home()

SetPositions: {'Z': 0}
Move: {'Z': 30000}
homing: [<valurap.axes.AxeX1 object at 0xab585190>, <valurap.axes.AxeX2 object at 0xab585050>, <valurap.axes.AxeY object at 0xab5851f0>]
Warning, explicit X in segment
Warning, explicit X in segment
Warning, explicit X in segment
SetPositions: {'X1': 0, 'X2': 0, 'Y': 0}
homing: [<valurap.axes.AxeX1 object at 0xab585190>, <valurap.axes.AxeX2 object at 0xab585050>, <valurap.axes.AxeY object at 0xab5851f0>]
Warning, explicit X in segment
X2 already on endstop
Warning, explicit X in segment
SetPositions: {'X1': 0, 'Y': 0}
homing: [<valurap.axes.AxeX1 object at 0xab585190>, <valurap.axes.AxeX2 object at 0xab585050>, <valurap.axes.AxeY object at 0xab5851f0>]
X1 already on endstop
X2 already on endstop
Warning, explicit X in segment
SetPositions: {'Y': 0}
homing: [<valurap.axes.AxeX1 object at 0xab585190>, <valurap.axes.AxeX2 object at 0xab585050>, <valurap.axes.AxeY object at 0xab5851f0>]
X1 already on endstop
X2 already on endstop
Y already on endst

In [3]:
prn.update_axes_positions()

prn.axe_e1.enabled = True
prn.axe_x1.enabled = True
prn.axe_y.enabled = True
prn.axe_z.enabled = True
prn.update_axes_config()

prn.moveto(X1=0)
prn.moveto(Y=0)

Save position: X1 12698.84765625
Save position: X2 -13501.1484375
Save position: Y -20001.1484375
SetPositions: {'X1': 12698.84765625, 'X2': -13501.1484375, 'Y': -20001.1484375}
MoveTo: {'X1': 0}
Save position: X1 12697.84765625
Save position: X2 -13502.1484375
Save position: Y -20002.1484375
Move: {'X1': -12697.84765625}
MoveTo: {'Y': 0}
Save position: X1 -0.4140625
Move: {'Y': 20002.1484375}


In [ ]:
#prn.moveto(X1=0)
#prn.moveto(Y=0)

#prn.move(Z=-1600*30)
x1, y1 = [255.1652, 288.87604]
x2, y2 = [339.1803, 288.58218]

x3, y3 = [339.1803, 288.58218]
x4, y4 = [339.62244, 372.909]

x, y = [358.5733, 240.30933]


dx = (x - 320)/(x2 - x1)*10
dy = (y - 240)/(y4 - y3)*10
display(dx, dy)
z1 = [
    [320.05545, 240.03896],
    [320.58078, 243.7491],
    [321.13934, 247.78864],
    [321.85776, 252.33138]
]

dx = -40
dy = 0
prn.move(X2=-80*dx, Y=-80*dy)
#prn.move(Z=-1600*30)
#prn.move(X2=-80*10)


In [9]:
if 0:
    prn.axe_e1.enabled = True
    prn.axe_x1.enabled = True
    prn.axe_y.enabled = True
    prn.axe_z.enabled = True
    prn.axe_e2.enabled = False
    prn.update_axes_config()

#prn.move(E1=-100000)
#prn.move(E1=-20000)
#prn.moveto(X1=1000)
#prn.moveto(X2=0)
#prn.moveto(Y=-3000)
prn.moveto(Z=5040)
prn.moveto(Z=5040-1600*(0.7-0.3))

MoveTo: {'Z': 5040}
Save position: Z 3440.296875
Move: {'Z': 1599.703125}
MoveTo: {'Z': 4400.0}
Save position: Z 5039.29296875
Move: {'Z': -639.29296875}


In [ ]:
prn.axe_e2.max_v=20000*3
prn.axe_e1.max_v=20000*22
display(22/3)
display(3*3/1.75/1.75)

In [ ]:
display(prn.axe_x1.last_pos)
display(prn.axe_x2.last_pos)
display(prn.axe_y.last_pos)

In [2]:
# open-loop
import valurap.asg as vas
from valurap import path_planning, emulate
import pickle
import imp

imp.reload(vas)
imp.reload(path_planning)

spm = 80
spme = 847
acc_step = 10000

apgs = {
    "X": prn.apg_x,
    "Y": prn.apg_y,
    "Z": prn.apg_z,
}

codes = []

up_segs = path_planning.ext_to_code(0.3, 100, spm=1600, max_a=1000)
up_code = prn.asg.gen_path_code(up_segs, accel_step=int(50000000/acc_step), real_apgs=apgs)

for axe in prn.axes.values():
    axe.apg = None
    axe.enabled = False

prn.axe_z.enabled = True
prn.axe_y.enabled = True
prn.axe_x1.enabled = True
prn.axe_e1.enabled = True
prn.update_axes_config()

start = None
apg_map = None

for layer in range(1, 3000):
    with open("test22_{:05d}.layer".format(layer), "rb") as f:
        p = pickle.load(f)

    if len(p) == 1 and p[0][0] == "do_home":
        break
        
    assert len(p) == 2
    assert p[0][0] == "start"
    assert p[1][0] == "segment"
    if start is None:
        start = p[0]

    pr_opt = [
        [1, [
            vas.ProfileSegment(apgs["X"], x=p[0][1]["X"] * spm),
            vas.ProfileSegment(apgs["Y"], x=p[0][1]["Y"] * spm),
            vas.ProfileSegment(apgs["Z"], x=0),
        ]]
    ]
    cms, meta, segments = p[1]
    if apg_map is None:
        apg_map = meta["map"]
        
    codes.append(prn.asg.gen_map_code(meta["map"]))
    for dt, segs in segments:
        pr_opt.append([
            dt, [vas.ProfileSegment.from_tuple(s, apgs) for s in segs]
        ])

    acc_step = 10000
    path_code = prn.asg.gen_path_code(pr_opt, accel_step=int(50000000/acc_step), real_apgs={"X": prn.apg_x, "Y": prn.apg_y, "Z": prn.apg_z})
    print(len(path_code))
    codes.append(path_code)
    codes.append(prn.asg.gen_map_code({"Z": "Z"}))
    codes.append(up_code)

prn.update_axes_config()
fullcode = []
for code in codes:
    fullcode.extend(code[:-1])
fullcode.extend(code[-1:])

print(len(fullcode))

prn.setup()

for axe in prn.axes.values():
    axe.apg = None
    axe.enabled = False
    
prn.update_axes_config()
prn.home()

prn.update_axes_positions()

prn.axe_e1.enabled = True
prn.axe_x1.enabled = True
prn.axe_y.enabled = True
prn.axe_z.enabled = True
prn.axe_e2.enabled = False
prn.update_axes_config()

prn.move(E1=-100000)
#prn.move(E1=-20000)
prn.moveto(X1=0)
prn.moveto(Y=0)
#prn.moveto(X2=0)
prn.moveto(Y=start[1]["Y"] * 80, X1=start[1]["X"] * 80)
prn.moveto(Z=5040)
#prn.moveto(Z=5040-1600*1.5) # clear glass
prn.moveto(Z=5040-1600*(0.6-0.3))

for k, v in apg_map.items():
    prn.axes[k].apg = prn.apgs[v]
    
#print(start[2])
#prn.moveto(X1=start[1]["X"] * 80, Y=start[1]["Y"] * 80)

try:
    prn.exec_long_code(fullcode, splits=2000, verbose=True)
    
    for axe in prn.axes.values():
        axe.apg = None
        axe.enabled = False
    
    prn.update_axes_config()
    prn.home()
    prn.setup()
except:
    prn.setup()
    


Warning, explicit X in segment
Warning, explicit X in segment
Warning, explicit X in segment
Map: {'X1': 'X', 'Y': 'Y', 'E1': 'Z'}
Axe X1 mapped to X
Axe X1 enabled
Axe Y mapped to Y
Axe Y enabled
Axe E1 mapped to Z
Axe E1 enabled
Axe Z enabled
168293
Map: {'Z': 'Z'}
Axe X1 enabled
Axe Y enabled
Axe E1 enabled
Axe Z mapped to Z
Axe Z enabled
Warning, explicit X in segment
Warning, explicit X in segment
Warning, explicit X in segment
Map: {'X1': 'X', 'Y': 'Y', 'E1': 'Z'}
Axe X1 mapped to X
Axe X1 enabled
Axe Y mapped to Y
Axe Y enabled
Axe E1 mapped to Z
Axe E1 enabled
Axe Z enabled
177897
Map: {'Z': 'Z'}
Axe X1 enabled
Axe Y enabled
Axe E1 enabled
Axe Z mapped to Z
Axe Z enabled
Warning, explicit X in segment
Warning, explicit X in segment
Warning, explicit X in segment
Map: {'X1': 'X', 'Y': 'Y', 'E1': 'Z'}
Axe X1 mapped to X
Axe X1 enabled
Axe Y mapped to Y
Axe Y enabled
Axe E1 mapped to Z
Axe E1 enabled
Axe Z enabled
169269
Map: {'Z': 'Z'}
Axe X1 enabled
Axe Y enabled
Axe E1 enabled


Warning, explicit X in segment
Warning, explicit X in segment
Warning, explicit X in segment
Map: {'X1': 'X', 'Y': 'Y', 'E1': 'Z'}
Axe X1 mapped to X
Axe X1 enabled
Axe Y mapped to Y
Axe Y enabled
Axe E1 mapped to Z
Axe E1 enabled
Axe Z enabled
17978
Map: {'Z': 'Z'}
Axe X1 enabled
Axe Y enabled
Axe E1 enabled
Axe Z mapped to Z
Axe Z enabled
Warning, explicit X in segment
Warning, explicit X in segment
Warning, explicit X in segment
Map: {'X1': 'X', 'Y': 'Y', 'E1': 'Z'}
Axe X1 mapped to X
Axe X1 enabled
Axe Y mapped to Y
Axe Y enabled
Axe E1 mapped to Z
Axe E1 enabled
Axe Z enabled
17113
Map: {'Z': 'Z'}
Axe X1 enabled
Axe Y enabled
Axe E1 enabled
Axe Z mapped to Z
Axe Z enabled
Warning, explicit X in segment
Warning, explicit X in segment
Warning, explicit X in segment
Map: {'X1': 'X', 'Y': 'Y', 'E1': 'Z'}
Axe X1 mapped to X
Axe X1 enabled
Axe Y mapped to Y
Axe Y enabled
Axe E1 mapped to Z
Axe E1 enabled
Axe Z enabled
17857
Map: {'Z': 'Z'}
Axe X1 enabled
Axe Y enabled
Axe E1 enabled
Axe

14075
Map: {'Z': 'Z'}
Axe X1 enabled
Axe Y enabled
Axe E1 enabled
Axe Z mapped to Z
Axe Z enabled
Warning, explicit X in segment
Warning, explicit X in segment
Warning, explicit X in segment
Map: {'X1': 'X', 'Y': 'Y', 'E1': 'Z'}
Axe X1 mapped to X
Axe X1 enabled
Axe Y mapped to Y
Axe Y enabled
Axe E1 mapped to Z
Axe E1 enabled
Axe Z enabled
13283
Map: {'Z': 'Z'}
Axe X1 enabled
Axe Y enabled
Axe E1 enabled
Axe Z mapped to Z
Axe Z enabled
2117295
Executing reset code
Update axes
Reset OLED
Setup done
SetPositions: {'Z': 0}
Move: {'Z': 30000}
homing: [<valurap.axes.AxeX1 object at 0xab551f50>, <valurap.axes.AxeX2 object at 0xab551db0>, <valurap.axes.AxeY object at 0xab551e10>]
Warning, explicit X in segment
Warning, explicit X in segment
Warning, explicit X in segment
SetPositions: {'X1': 0, 'X2': 0, 'Y': 0}
homing: [<valurap.axes.AxeX1 object at 0xab551f50>, <valurap.axes.AxeX2 object at 0xab551db0>, <valurap.axes.AxeY object at 0xab551e10>]
X1 already on endstop
X2 already on endstop
Wa

free_buf: 2161
sent 2000 commands to addr 3616, left 2095295
Busy: 1 Wait: 1 Error: 2 PC: 15663 Cur addr: 5616 Cur len: 2095295
too small free_buf: 1855
Busy: 1 Wait: 1 Error: 2 PC: 16186 Cur addr: 5616 Cur len: 2095295
free_buf: 2378
sent 2000 commands to addr 5616, left 2093295
Busy: 1 Wait: 1 Error: 2 PC: 17941 Cur addr: 7616 Cur len: 2093295
free_buf: 2133
sent 2000 commands to addr 7616, left 2091295
Busy: 1 Wait: 1 Error: 2 PC: 19903 Cur addr: 1424 Cur len: 2091295
free_buf: 2095
sent 2000 commands to addr 1424, left 2089295
Busy: 1 Wait: 1 Error: 2 PC: 21652 Cur addr: 3424 Cur len: 2089295
too small free_buf: 1844
Busy: 1 Wait: 1 Error: 2 PC: 21920 Cur addr: 3424 Cur len: 2089295
free_buf: 2112
sent 2000 commands to addr 3424, left 2087295
Busy: 1 Wait: 1 Error: 2 PC: 23589 Cur addr: 5424 Cur len: 2087295
too small free_buf: 1781
Busy: 1 Wait: 1 Error: 2 PC: 23871 Cur addr: 5424 Cur len: 2087295
free_buf: 2063
sent 2000 commands to addr 5424, left 2085295
Busy: 1 Wait: 1 Error: 

too small free_buf: 1842
Busy: 1 Wait: 1 Error: 2 PC: 22324 Cur addr: 3888 Cur len: 2023295
free_buf: 2052
sent 2000 commands to addr 3888, left 2021295
Busy: 1 Wait: 1 Error: 2 PC: 24303 Cur addr: 5888 Cur len: 2021295
free_buf: 2031
sent 2000 commands to addr 5888, left 2019295
Busy: 1 Wait: 1 Error: 2 PC: 26245 Cur addr: 7888 Cur len: 2019295
too small free_buf: 1973
Busy: 1 Wait: 1 Error: 2 PC: 26685 Cur addr: 7888 Cur len: 2019295
free_buf: 2413
sent 2000 commands to addr 7888, left 2017295
Busy: 1 Wait: 1 Error: 2 PC: 27857 Cur addr: 1696 Cur len: 2017295
too small free_buf: 1585
Busy: 1 Wait: 1 Error: 2 PC: 27867 Cur addr: 1696 Cur len: 2017295
too small free_buf: 1595
Busy: 1 Wait: 1 Error: 2 PC: 28358 Cur addr: 1696 Cur len: 2017295
free_buf: 2086
sent 2000 commands to addr 1696, left 2015295
Busy: 1 Wait: 1 Error: 2 PC: 29982 Cur addr: 3696 Cur len: 2015295
too small free_buf: 1710
Busy: 1 Wait: 1 Error: 2 PC: 30475 Cur addr: 3696 Cur len: 2015295
free_buf: 2203
sent 2000 com

...

too small free_buf: 993
Busy: 1 Wait: 1 Error: 2 PC: 42927 Cur addr: 768 Cur len: 101295
too small free_buf: 1199
Busy: 1 Wait: 1 Error: 2 PC: 43043 Cur addr: 768 Cur len: 101295
too small free_buf: 1315
Busy: 1 Wait: 1 Error: 2 PC: 43179 Cur addr: 768 Cur len: 101295
too small free_buf: 1451
Busy: 1 Wait: 1 Error: 2 PC: 43305 Cur addr: 768 Cur len: 101295
too small free_buf: 1577
Busy: 1 Wait: 1 Error: 2 PC: 43451 Cur addr: 768 Cur len: 101295
too small free_buf: 1723
Busy: 1 Wait: 1 Error: 2 PC: 43569 Cur addr: 768 Cur len: 101295
too small free_buf: 1841
Busy: 1 Wait: 1 Error: 2 PC: 43876 Cur addr: 768 Cur len: 101295
free_buf: 2148
sent 2000 commands to addr 768, left 99295
Busy: 1 Wait: 1 Error: 2 PC: 45664 Cur addr: 2768 Cur len: 99295
too small free_buf: 1936
Busy: 1 Wait: 1 Error: 2 PC: 45808 Cur addr: 2768 Cur len: 99295
free_buf: 2080
sent 2000 commands to addr 2768, left 97295
Busy: 1 Wait: 1 Error: 2 PC: 46484 Cur addr: 4768 Cur len: 97295
too small free_buf: 756
Busy: 1 Wa

too small free_buf: 690
Busy: 1 Wait: 1 Error: 2 PC: 3024 Cur addr: 2192 Cur len: 75295
too small free_buf: 832
Busy: 1 Wait: 1 Error: 2 PC: 3252 Cur addr: 2192 Cur len: 75295
too small free_buf: 1060
Busy: 1 Wait: 1 Error: 2 PC: 3406 Cur addr: 2192 Cur len: 75295
too small free_buf: 1214
Busy: 1 Wait: 1 Error: 2 PC: 3647 Cur addr: 2192 Cur len: 75295
too small free_buf: 1455
Busy: 1 Wait: 1 Error: 2 PC: 3860 Cur addr: 2192 Cur len: 75295
too small free_buf: 1668
Busy: 1 Wait: 1 Error: 2 PC: 4074 Cur addr: 2192 Cur len: 75295
too small free_buf: 1882
Busy: 1 Wait: 1 Error: 2 PC: 4196 Cur addr: 2192 Cur len: 75295
too small free_buf: 2004
Busy: 1 Wait: 1 Error: 2 PC: 4399 Cur addr: 2192 Cur len: 75295
free_buf: 2207
sent 2000 commands to addr 2192, left 73295
Busy: 1 Wait: 1 Error: 2 PC: 5236 Cur addr: 4192 Cur len: 73295
too small free_buf: 1044
Busy: 1 Wait: 1 Error: 2 PC: 5425 Cur addr: 4192 Cur len: 73295
too small free_buf: 1233
Busy: 1 Wait: 1 Error: 2 PC: 5559 Cur addr: 4192 Cur 

too small free_buf: 1278
Busy: 1 Wait: 1 Error: 2 PC: 27611 Cur addr: 1616 Cur len: 51295
too small free_buf: 1419
Busy: 1 Wait: 1 Error: 2 PC: 27771 Cur addr: 1616 Cur len: 51295
too small free_buf: 1579
Busy: 1 Wait: 1 Error: 2 PC: 28422 Cur addr: 1616 Cur len: 51295
free_buf: 2230
sent 2000 commands to addr 1616, left 49295
Busy: 1 Wait: 1 Error: 2 PC: 29768 Cur addr: 3616 Cur len: 49295
too small free_buf: 1576
Busy: 1 Wait: 1 Error: 2 PC: 29919 Cur addr: 3616 Cur len: 49295
too small free_buf: 1727
Busy: 1 Wait: 1 Error: 2 PC: 30065 Cur addr: 3616 Cur len: 49295
too small free_buf: 1873
Busy: 1 Wait: 1 Error: 2 PC: 30221 Cur addr: 3616 Cur len: 49295
free_buf: 2029
sent 2000 commands to addr 3616, left 47295
Busy: 1 Wait: 1 Error: 2 PC: 31105 Cur addr: 5616 Cur len: 47295
too small free_buf: 913
Busy: 1 Wait: 1 Error: 2 PC: 31326 Cur addr: 5616 Cur len: 47295
too small free_buf: 1134
Busy: 1 Wait: 1 Error: 2 PC: 31926 Cur addr: 5616 Cur len: 47295
too small free_buf: 1734
Busy: 1 

too small free_buf: 1518
Busy: 1 Wait: 1 Error: 2 PC: 55858 Cur addr: 5040 Cur len: 23295
too small free_buf: 1666
Busy: 1 Wait: 1 Error: 2 PC: 55995 Cur addr: 5040 Cur len: 23295
too small free_buf: 1803
Busy: 1 Wait: 1 Error: 2 PC: 56155 Cur addr: 5040 Cur len: 23295
too small free_buf: 1963
Busy: 1 Wait: 1 Error: 2 PC: 56263 Cur addr: 5040 Cur len: 23295
free_buf: 2071
sent 2000 commands to addr 5040, left 21295
Busy: 1 Wait: 1 Error: 2 PC: 57605 Cur addr: 7040 Cur len: 21295
too small free_buf: 1413
Busy: 1 Wait: 1 Error: 2 PC: 58190 Cur addr: 7040 Cur len: 21295
too small free_buf: 1998
Busy: 1 Wait: 1 Error: 2 PC: 58341 Cur addr: 7040 Cur len: 21295
free_buf: 2149
sent 2000 commands to addr 7040, left 19295
Busy: 1 Wait: 1 Error: 2 PC: 58441 Cur addr: 848 Cur len: 19295
too small free_buf: 249
Busy: 1 Wait: 1 Error: 2 PC: 58451 Cur addr: 848 Cur len: 19295
too small free_buf: 259
Busy: 1 Wait: 1 Error: 2 PC: 58466 Cur addr: 848 Cur len: 19295
too small free_buf: 274
Busy: 1 Wait:

Busy: 1 Wait: 1 Error: 2 PC: 16033 Cur addr: 3759 Cur len: 0
Busy: 1 Wait: 1 Error: 2 PC: 16308 Cur addr: 3759 Cur len: 0
Busy: 1 Wait: 1 Error: 2 PC: 16605 Cur addr: 3759 Cur len: 0
Busy: 1 Wait: 1 Error: 2 PC: 16853 Cur addr: 3759 Cur len: 0
Busy: 1 Wait: 1 Error: 2 PC: 17041 Cur addr: 3759 Cur len: 0
Busy: 1 Wait: 1 Error: 2 PC: 17379 Cur addr: 3759 Cur len: 0
Busy: 1 Wait: 1 Error: 2 PC: 17676 Cur addr: 3759 Cur len: 0
Busy: 1 Wait: 1 Error: 2 PC: 17901 Cur addr: 3759 Cur len: 0
Busy: 1 Wait: 1 Error: 2 PC: 17944 Cur addr: 3759 Cur len: 0
Busy: 1 Wait: 1 Error: 2 PC: 18146 Cur addr: 3759 Cur len: 0
Busy: 1 Wait: 1 Error: 2 PC: 18458 Cur addr: 3759 Cur len: 0
Busy: 1 Wait: 1 Error: 2 PC: 18757 Cur addr: 3759 Cur len: 0
Busy: 1 Wait: 1 Error: 2 PC: 19078 Cur addr: 3759 Cur len: 0
Busy: 1 Wait: 1 Error: 2 PC: 19387 Cur addr: 3759 Cur len: 0
Busy: 1 Wait: 1 Error: 2 PC: 19664 Cur addr: 3759 Cur len: 0
Busy: 1 Wait: 1 Error: 2 PC: 19963 Cur addr: 3759 Cur len: 0
Busy: 1 Wait: 1 Error: 2

In [ ]:
# check

import valurap.asg as vas
from valurap import path_planning, emulate
import pickle
import imp

imp.reload(vas)
imp.reload(path_planning)
imp.reload(emulate)

spm = 80
spme = 847
acc_step = 10000

apgs = {
    "X": prn.apg_x,
    "Y": prn.apg_y,
    "Z": prn.apg_z,
}

codes = []

up_segs = path_planning.ext_to_code(0.3, 100, spm=1600, max_a=1000)
up_code = prn.asg.gen_path_code(up_segs, accel_step=int(50000000/acc_step), real_apgs={"X": prn.apg_x, "Y": prn.apg_y, "Z": prn.apg_z})

for axe in prn.axes.values():
    axe.apg = None
    axe.enabled = False

if 1:
    prn.axe_z.enabled = True
    prn.axe_y.enabled = True
    prn.axe_x1.enabled = True
    prn.axe_e1.enabled = True

apg_states = {}
codes = []
first = True
for layer in range(1, 100):
    with open("test5_{:05d}.layer".format(layer), "rb") as f:
        p = pickle.load(f)

    assert len(p) == 2
    assert p[0][0] == "start"
    assert p[1][0] == "segment"
    
    print(p[0])
    expected_pos = p[0][1]

    if apg_states:
        print(expected_pos["X"] - apg_states["X"].x / 2**32 / spm)
        print(expected_pos["Y"] - apg_states["Y"].x / 2**32 / spm)

    cms, meta, segments = p[1]
    code = prn.asg.gen_map_code(meta["map"])[:-1]

    pr_opt = []
    
    if first:
        pr_opt += [
            [1, [
                vas.ProfileSegment(apgs["X"], x=int(p[0][1]["X"] * spm)),
                vas.ProfileSegment(apgs["Y"], x=int(p[0][1]["Y"] * spm)),
                vas.ProfileSegment(apgs["Z"], x=0),
            ]]
        ]
        first = False

    
    for dt, segs in segments:
        pr_opt.append([
            dt, [vas.ProfileSegment.from_tuple(s, apgs) for s in segs]
        ])

    if 0:
        pr_opt += [
            [1, [
                vas.ProfileSegment(apgs["X"], v=0),
                vas.ProfileSegment(apgs["Y"], v=0),
                vas.ProfileSegment(apgs["Z"], v=0),
            ]]
        ]

    acc_step = 10000
    path_code = prn.asg.gen_path_code(pr_opt, accel_step=int(50000000/acc_step), real_apgs={"X": prn.apg_x, "Y": prn.apg_y, "Z": prn.apg_z})
    code += path_code[:-1]
    print(len(path_code))
    
    emulate.emulate(pr_opt, accel_step=50000000/acc_step, verbose=0, apg_states=apg_states, no_tracking=True)
    emu_x = int(apg_states["X"].x) & 0xffffffffff000000
    emu_y = int(apg_states["Y"].x) & 0xffffffffff000000
    
    code += prn.asg.gen_map_code({"Z": "Z"})[:-1]
    code += up_code
    codes.append([expected_pos, (emu_x, emu_y), code])

prn.update_axes_config()

deltas = []
try:
    for exp_pos, (exp_x, exp_y), code in codes:
        prn.exec_long_code(code, splits=2000, verbose=True)
        
        cur_x_hi = prn.s3g.S3G_INPUT(prn.apg_x.cur_x_hi)
        cur_y_hi = prn.s3g.S3G_INPUT(prn.apg_y.cur_x_hi)
        cur_x_lo = prn.s3g.S3G_INPUT(prn.apg_x.cur_x_lo)
        cur_y_lo = prn.s3g.S3G_INPUT(prn.apg_y.cur_x_lo)

        if cur_x_hi < 0:
            cur_x_hi += 2**32

        if cur_y_hi < 0:
            cur_y_hi += 2**32

        if cur_x_lo < 0:
            cur_x_lo += 2**32

        if cur_y_lo < 0:
            cur_y_lo += 2**32
            
        cur_x = cur_x_hi << 32 | cur_x_lo
        cur_y = cur_y_hi << 32 | cur_y_lo

        
        if 0:
            if exp_x > 2 ** 63:
                exp_x -= 2 ** 63

            if exp_y > 2 ** 63:
                exp_y -= 2 ** 63

            if cur_x > 2 ** 63:
                cur_x -= 2 ** 63

            if cur_y > 2 ** 63:
                cur_y -= 2 ** 63

            exp_x /= 2 ** 32 * spm
            exp_y /= 2 ** 32 * spm
            cur_x /= 2 ** 32 * spm
            cur_y /= 2 ** 32 * spm

        print("AAAAAAAA", exp_x, exp_y)
        print("AAAAAAAA", cur_x, cur_y)
        deltas.append([exp_x, cur_x, exp_y, cur_y])

except:
    prn.setup()
    print(deltas)
    raise

    
print(deltas)

In [58]:
for axe in prn.axes.values():
    axe.apg = None
    axe.enabled = False
    
prn.update_axes_config()
prn.home()

SetPositions: {'Z': 5038.6796875}
Move: {'Z': 30000}
Save position: Z 5037.6796875
homing: [<valurap.axes.AxeX1 object at 0xa8732890>, <valurap.axes.AxeX2 object at 0xa87328f0>, <valurap.axes.AxeY object at 0xa87328b0>]
Warning, explicit X in segment
Warning, explicit X in segment
Warning, explicit X in segment
SetPositions: {'X1': 1862.89453125, 'X2': -13503.1484375, 'Y': -1845.66796875}
homing: [<valurap.axes.AxeX1 object at 0xa8732890>, <valurap.axes.AxeX2 object at 0xa87328f0>, <valurap.axes.AxeY object at 0xa87328b0>]
Warning, explicit X in segment
X2 already on endstop
Warning, explicit X in segment
SetPositions: {'X1': 1862.89453125, 'Y': -1845.66796875}
homing: [<valurap.axes.AxeX1 object at 0xa8732890>, <valurap.axes.AxeX2 object at 0xa87328f0>, <valurap.axes.AxeY object at 0xa87328b0>]
X1 already on endstop
X2 already on endstop
Warning, explicit X in segment
SetPositions: {'Y': -1845.66796875}
homing: [<valurap.axes.AxeX1 object at 0xa8732890>, <valurap.axes.AxeX2 object at 

In [65]:
prn.moveto(X1=-13600)

MoveTo: {'X1': -13600}
Save position: X1 -15000.0546875
Move: {'X1': 1400.0546875}


In [ ]:
from valurap import emulate

apgs = {
    "X": prn.apg_x,
    "Y": prn.apg_y,
    "Z": prn.apg_z,
}

for axe in prn.axes.values():
    axe.apg = None
    axe.enabled = False

if 0:
    prn.axe_z.enabled = True
    prn.axe_y.enabled = True
    prn.axe_x1.enabled = True
    prn.axe_e1.enabled = True
    
pr_opt = []
    
pr_opt += [
    [1, [
        vas.ProfileSegment(apgs["X"], x=0),
        vas.ProfileSegment(apgs["Y"], x=0),
        vas.ProfileSegment(apgs["Z"], x=0),
    ]],
    [1000, [
        vas.ProfileSegment(apgs["X"], v=-1000000, a=1000000, j=-10000),
        vas.ProfileSegment(apgs["Y"], v=-1000000, a=1000000, j=-10001),
        vas.ProfileSegment(apgs["Z"], v=0),
    ]],
    [1000, [
        vas.ProfileSegment(apgs["X"], a=-1000000, j=11000),
        vas.ProfileSegment(apgs["Y"], a=-1000000, j=11001),
        vas.ProfileSegment(apgs["Z"], v=0),
    ]],
    [1, [
        vas.ProfileSegment(apgs["X"], v=0),
        vas.ProfileSegment(apgs["Y"], v=0),
        vas.ProfileSegment(apgs["Z"], v=0),
    ]],
]

acc_step = 10000
code = prn.asg.gen_path_code(pr_opt, accel_step=int(50000000/acc_step), real_apgs=apgs)
prn.exec_long_code(code, splits=2000, verbose=True)
cur_x = prn.s3g.S3G_INPUT(prn.apg_x.cur_x_hi)
cur_y = prn.s3g.S3G_INPUT(prn.apg_y.cur_x_hi)
cur_x_lo = prn.s3g.S3G_INPUT(prn.apg_x.cur_x_lo)
cur_y_lo = prn.s3g.S3G_INPUT(prn.apg_y.cur_x_lo)

if cur_x < 0:
    cur_x += 2**32

if cur_y < 0:
    cur_y += 2**32

if cur_x_lo < 0:
    cur_x_lo += 2**32

if cur_y_lo < 0:
    cur_y_lo += 2**32

apg_states = {}
res = emulate.emulate(pr_opt, accel_step=50000000/acc_step, verbose=0, apg_states=apg_states)

print(cur_x << 32 | cur_x_lo)
print(cur_y << 32 | cur_y_lo)
print(int(apg_states["X"].x) & 0xffffffffff000000)
print(int(apg_states["Y"].x) & 0xffffffffff000000)